# PointNet
## 概要
PointNet[Qi et al. 2017a]は点群から点ごとの特徴(pointwise feature)と点群全体の特徴(global feature)を抽出することが可能なネットワークである。PointNetアーキテクチャは以下の通り。

![pointnet](img/pointnet.png)

PointNetは点群をそのままネットワーク上で処理可能にするため、以下の問題に対して上記の構造を提案した(点群の問題については[characteristic.ipynb](characteristic.ipynb))。

- **点の順不同性**: 点は規則的に並んでいない場合があるため、画像のように要素インデックスを参照した畳み込みによる特徴抽出を行うことはできない。そこで、本提案では各点で畳み込みが可能なPoint-wise convolution layer (図のmlp)を採用することで個々の点が持つ特徴のみを畳みこむ。また、点群全体の特徴量を一つの特徴(global feature)にpoolingする際、ここでも点の順不同性を考える必要があるため、畳みこまれた特徴に対するPoolingにはSymmetric Functionを使う。
- **剛体変換されたオブジェクトに対する予測の不変性**: 入力されるオブジェクト点群が全て決まった一方向に向いていない可能性がある。深層学習モデルにとっても、オブジェクトの向きはランダムではない方が好ましい。そこで、深層学習モデルにとって予測しやすい方向にオブジェクト点群を変換するSpatial transformer networks [Jaderberg et al. 2015] (図中ではT-Net)を使用する。尚、本提案ではオブジェクト点群と特徴量空間上の特徴に対して適用している。

本資料では、図中の上のClassificationモデルの実装を紹介する。

## 処理手順
classificationのための処理の手順は以下の通り。

1. サンプリング
   1. 各オブジェクトのメッシュデータから、点群をサンプリングする。
   2. 点群の点は任意の個数Nだけサンプリングされる。また、各点にはXYZ座標値が入る。
2. PointNetへの入力
   1. PointNetで予測を行う際は、バッチサイズB個分のオブジェクトを用意する。
   2. この時、追加データのサイズをPointNetに入力するデータの形状は[B, 3, N] (pytorchフォーマット)となる。3はXYZ座標値の次元数である。
3. PointNetからの出力
   1. PointNetから予測結果とT-Netのtransformation matrixが出力される。
   2. 予測結果は[B]の形状を持つ配列としてPointNetから出力される。
   3. transformation matrixは3x3配列と64x64配列の2種類出力される。
   4. [訓練時]: 訓練では、予測結果に対するCross Entropy損失、64x64のtransformation matrixに対する正則化項(論文内では$L_reg$)を損失計算に使う。


## 本レポジトリ以外の実装例
### [charlesq34/pointnet](https://github.com/charlesq34/pointnet)
- 本家のtensorflow実装

### [yanx27/Pointnet_Pointnet2_pytorch](https://github.com/yanx27/Pointnet_Pointnet2_pytorch)
- pytorch実装
- Classificationについて、精度向上に向けた工夫がある。工夫は以下の通り。
  - normal (法線情報)を点の特徴量に入れた点群を用意し、それをPointNetに入力する。
  - 入力するオブジェクト点群を、0~1に正規化する。公式実装では、0~1に正規化されていない。
- Overall accuracyは89.2から91.4に向上した。

### [fxia22/pointnet.pytorch](https://github.com/fxia22/pointnet.pytorch)
- pytorch実装
- 本家実装よりもClassificationのOverall accuracyが低め。
  - 本家実装では、Batch normalizationのパラメータに対するスケジューリングのような少々pytorchで実装しづらいと考えられる箇所があるため、その違いによって精度の差が出た可能性がある。
- feature transformation (64x64のT-Net)の有無による精度比較も行っている。

## 訓練
PointNetを用いた訓練は以下のpointnet.pyを介して実行できる。尚、以下のコードを本ファイルに書き込まない理由は、Jupyter上でのGPUのメモリ解放が厄介であるためである。

In [4]:
!python pointnet.py --mode=train --device=0 --output_dir_path=outputs/pointnet/ --dataset_dir_path=../../data/modelnet40_ply_hdf5_2048/

train: 100%|██████████████| 250/250 [47:31<00:00, 11.41s/it]


## テスト
PointNetを用いたテストは以下のpointnet.pyを介して実行できる。尚、以下のコードを本ファイルに書き込まない理由は、Jupyter上でのGPUのメモリ解放が厄介であるためである。


In [ ]:
!python pointnet.py --mode=test --device=0 --output_dir_path=outputs/pointnet/ --dataset_dir_path=../../data/modelnet40_ply_hdf5_2048/ --test_model_file_path=outputs/pointnet/model.pth

## References
- Qi, Charles R., Hao Su, Kaichun Mo, and Leonidas J. Guibas. 2017. “Pointnet: Deep Learning on Point Sets for 3d Classification and Segmentation.” In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, 652–60.
- Jaderberg, Max, Karen Simonyan, Andrew Zisserman, and Koray Kavukcuoglu. 2015. “Spatial Transformer Networks.” arXiv [cs.CV]. arXiv. http://arxiv.org/abs/1506.02025.